In [2]:
%matplotlib notebook
import sys
from PyQt5 import QtCore, QtWidgets, QtGui
from PyQt5.QtWidgets import *
from PyQt5.QtCore import Qt
import threading
from functools import partial

In [13]:
class SIM928:
    def __init__(self, name):
        self.name = name
        self.val = 0
        self.scale = 8
        
    def __call__(self, *args):
        if len(args) == 0:
            return self.val
        else:
            self.val = args[0]

def create_window(window, *args):
    qApp = QtWidgets.QApplication(sys.argv)
    aw = window(*args)
    aw.show()
    qApp.exec_()

class SIM928Dialog(QFrame):
    def __init__(self, parameter, idx=1):
        super().__init__()
        
        self.parameter = parameter
        self.idx = idx
        
        self._state = 'none'
        self.step = 0.001
        
#         self.setGeometry(300, 300, 120, 200)
        self.setMaximumWidth(200)
#         self.setFrameShape(QFrame.StyledPanel)
    
        self.initUI()

    @property
    def state(self):
        return self._state
    
    @state.setter
    def state(self, state):
        if state not in ['none', 'up_down', 'left_right']:
            print(f'State {state} not recognized')
            
        else:
            # Create palette for name color customization
            name_palette = QtGui.QPalette()
        if state == 'none':
            name_palette.setColor(QtGui.QPalette.Foreground,QtCore.Qt.black)
        elif state == 'up_down':
            name_palette.setColor(QtGui.QPalette.Foreground,QtCore.Qt.blue)
        elif state == 'left_right':
            name_palette.setColor(QtGui.QPalette.Foreground,QtCore.Qt.green)
        self._state = state
        self.name_label.setPalette(name_palette)
        
    def cycle_state(self, *args, **kwargs):
        if self.state == 'none':
            self.state = 'up_down'
        elif self.state == 'up_down':
            self.state = 'left_right'
        else:
            self.state = 'none'
        
    def initUI(self):
        layout = QVBoxLayout()
        self.setLayout(layout)
        
        
        # Set parameter name
        self.name_label = QLabel(f'{self.idx}: {self.parameter.name}')
        self.name_label.setAlignment(Qt.AlignCenter)
        self.name_label.setFont(QtGui.QFont("Times", 16, QtGui.QFont.Bold) )
        self.name_label.mousePressEvent = self.cycle_state
        layout.addWidget(self.name_label)

        
        # Add step size
        step_hbox = QHBoxLayout()
        step_hbox.addStretch(0.8)
        step_hbox.addWidget(QLabel('Step:'))
        self.step_textbox = QLineEdit(str(self.step))
        self.step_textbox.setMaximumWidth(55)
        step_hbox.addWidget(self.step_textbox)
        self.step_textbox.returnPressed.connect(
            lambda: self.set_step(self.step_textbox.text()))
        step_hbox.addWidget(QLabel('V  '))
        layout.addLayout(step_hbox)
        
        
        # Add editable voltage
        val_descr_label = QLabel('Val:')
        val_descr_label.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold) )
        
        self.val_textbox = QLineEdit('{:.5g}'.format(self.parameter()))
        self.val_textbox.setAlignment(Qt.AlignCenter)
        self.val_textbox.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold) )
        self.val_textbox.returnPressed.connect(
            lambda: self.set_voltage(self.val_textbox.text()))
        
        val_units_label = QLabel('V')
        val_units_label.setFont(QtGui.QFont("Times", 12, QtGui.QFont.Bold) )

        val_hbox = QHBoxLayout()
        val_hbox.addWidget(val_descr_label)
        val_hbox.addWidget(self.val_textbox)
        val_hbox.addWidget(val_units_label)
        layout.addLayout(val_hbox)

        
        # Add current val
        self.current_val_label = QLabel(f'')
        self.current_val_label.setAlignment(Qt.AlignCenter)
        self.current_val_label.setFont(QtGui.QFont("Times", 12) )
        layout.addWidget(self.current_val_label)
        
        self.val_textbox.textEdited.connect(self._val_textbox_changed)

        
        # Add voltage buttons
        self.val_grid = QGridLayout()
        self.val_grid.setHorizontalSpacing(0)
        self.val_grid.setVerticalSpacing(0)
        layout.addLayout(self.val_grid)
        self.val_buttons = {}
        for column_idx, scale in enumerate([100, 10, 1]):
            for row_idx, sign in enumerate([1, -1]):
                val = sign * scale 
                button = QPushButton(f"{'+' if sign == 1 else '-'}{scale} mV")
                self.val_grid.addWidget(button, row_idx, column_idx+1)
                width = button.fontMetrics().boundingRect(f"+100 mV").width() + 7
                button.setMaximumWidth(width)
                button.clicked.connect(partial(self.increase_voltage, val/1000))
                self.val_buttons[val] = button
        

        # Add scale
        self.scale_label = QLabel(f'Scale: {self.parameter.scale}')
        self.scale_label.setAlignment(Qt.AlignCenter)
        self.scale_label.setFont(QtGui.QFont("Times", 12) )
        layout.addWidget(self.scale_label)
        
        
        # Add SIM voltage
        self.raw_val_label = QLabel('SIM val: {:.3g} V'.format(
            self.parameter() * self.parameter.scale))
        self.raw_val_label.setAlignment(Qt.AlignCenter)
        self.raw_val_label.setFont(QtGui.QFont("Times", 12) )
        layout.addWidget(self.raw_val_label)

    def _val_textbox_changed(self):
        self.val_textbox.setStyleSheet("color: rgb(255, 0, 0);")
        if self.current_val_label.text() == '':
            self.current_val_label.setText(f'Current val: {self.parameter()}')
        
    def set_step(self, val):
        try:
            val = round(float(val), 6)
            self.step = val
            self.clearFocus()
        except:
            pass
        
    def set_voltage(self, val):
        try:
            val = round(float(val), 6)
            self.parameter(val)
            self.raw_val_label.setText('SIM val: {:.3g} V'.format(
                self.parameter() * self.parameter.scale))
        except:
            pass
        self._reset_val_textbox()
        
    def _reset_val_textbox(self):
        self.val_textbox.setText('{:.5g}'.format(self.parameter()))
        self.current_val_label.setText('')
        self.val_textbox.setStyleSheet("color: rgb(0, 0, 0);")

    def increase_voltage(self, increment):
        self.set_voltage(self.parameter() + increment)
            
            
class SIMDialog(QDialog):
    def __init__(self, parameters):
        super().__init__()
        self.parameters = parameters
        
        self.index_keys = {}
        
        self.layout = QHBoxLayout()
        self.layout.setSpacing(0)
        self.setLayout(self.layout)
        for k, parameter in enumerate(self.parameters):
            separator = QFrame()
            separator.setFrameShape(QFrame.VLine)
            separator.setSizePolicy(QSizePolicy.Minimum,QSizePolicy.Expanding)
            separator.setLineWidth(2)
            self.layout.addWidget(separator)
        
            SIM928_dialog = SIM928Dialog(parameter, idx=k+1)
            self.layout.addWidget(SIM928_dialog)
            Qt_index_key = getattr(Qt, f'Key_{k+1}')
            self.index_keys[Qt_index_key] = SIM928_dialog
            
    def keyPressEvent(self, event): 
        try:
            if event.key() in self.index_keys:
                self.index_keys[event.key()].cycle_state()
            elif event.key() == Qt.Key_Escape:
                self._clear_focus()
#             elif event.key() == Qt.Key_Up:
#                 self.increase_voltage(self.step)
#             elif event.key() == Qt.Key_Down:
#                 self.increase_voltage(self.step)
        except Exception as e:
            print(f'error: {e}')
            
    def _clear_focus(self):
            focus_widget = QApplication.focusWidget()
            if focus_widget is not None:
                focus_widget.clearFocus()
#             if focus_widget == self.val_textbox:
#                 self._reset_val_textbox()
            
parameters = [SIM928('TGAC'), SIM928('DF'), SIM928('DS')]
parameter = parameters[0]
t = threading.Thread(target=create_window, name='gui', args=(SIMDialog, parameters))
t.start()

In [70]:
parameters = [SIM928('TGAC'), SIM928('DF'), SIM928('DS')]
parameter = parameters[0]
t = threading.Thread(target=create_window, name='gui', args=(SIM928Dialog, parameter))
t.start()

In [80]:
parameters = [SIM928('TGAC'), SIM928('DF'), SIM928('DS')]
parameter = parameters[0]
t = threading.Thread(target=create_window, name='gui', args=(SIMDialog, parameters))
t.start()

## Other examples

In [ ]:

class ApplicationWindow(QtWidgets.QMainWindow):
    def __init__(self, parameters):
        QtWidgets.QMainWindow.__init__(self)
        
         
        
        self.setAttribute(QtCore.Qt.WA_DeleteOnClose)
        self.setWindowTitle("application main window")
        self.main_widget = QtWidgets.QWidget(self)
        
        self.grid = QtWidgets.QGridLayout(self)
        self.setLayout(self.grid)
        
        self.parameters = []
        for parameter in self.parameters:
            self.add_DC_voltage(parameter)

        self.main_widget.setFocus()
        centralWidget = QtWidgets.QWidget()
        self.setCentralWidget(centralWidget)

    def add_DC_voltage(self, parameter):
        name_label = QtWidgets.QPushButton(parameter.name)
        position = (0, len(self.parameters))
        self.grid.addWidget(name_label, *position)
        self.parameters.append(parameter)
        
    def fileQuit(self):
        self.close()

    def closeEvent(self, ce):
        self.fileQuit()
        
class GridExample(QWidget):
    
    def __init__(self):
        super().__init__()
        
        self.initUI()
        
        
    def initUI(self):
        
        grid = QGridLayout()
        self.setLayout(grid)
 
        names = ['Cls', 'Bck', '', 'Close',
                 '7', '8', '9', '/',
                '4', '5', '6', '*',
                 '1', '2', '3', '-',
                '0', '.', '=', '+']
        
        positions = [(i,j) for i in range(5) for j in range(4)]
        
        for position, name in zip(positions, names):
            
            if name == '':
                continue
            button = QPushButton(name)
            grid.addWidget(button, *position)
            
        self.move(300, 150)
        self.setWindowTitle('Calculator')
        self.show()

# PyQT Matplotlib example

In [2]:
# embedding_in_qt5.py --- Simple Qt5 application embedding matplotlib canvases
#
# Copyright (C) 2005 Florent Rougon
#               2006 Darren Dale
#               2015 Jens H Nielsen
#
# This file is an example program for matplotlib. It may be used and
# modified with no restriction; raw copies as well as modified versions
# may be distributed without limitation.

from __future__ import unicode_literals
import sys
import os
import random
import matplotlib
# Make sure that we are using QT5
# matplotlib.use('Qt5Agg')
from PyQt5 import QtCore, QtWidgets

from numpy import arange, sin, pi
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

progname = os.path.basename(sys.argv[0])
progversion = "0.1"


class MyMplCanvas(FigureCanvas):
    """Ultimately, this is a QWidget (as well as a FigureCanvasAgg, etc.)."""

    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        # We want the axes cleared every time plot() is called
        self.axes.hold(False)

        self.compute_initial_figure()

        #
        FigureCanvas.__init__(self, fig)
        self.setParent(parent)

        FigureCanvas.setSizePolicy(self,
                                   QtWidgets.QSizePolicy.Expanding,
                                   QtWidgets.QSizePolicy.Expanding)
        FigureCanvas.updateGeometry(self)

    def compute_initial_figure(self):
        pass


class MyStaticMplCanvas(MyMplCanvas):
    """Simple canvas with a sine plot."""

    def compute_initial_figure(self):
        t = arange(0.0, 3.0, 0.01)
        s = sin(2*pi*t)
        self.axes.plot(t, s)


class MyDynamicMplCanvas(MyMplCanvas):
    """A canvas that updates itself every second with a new plot."""

    def __init__(self, *args, **kwargs):
        MyMplCanvas.__init__(self, *args, **kwargs)
        timer = QtCore.QTimer(self)
        timer.timeout.connect(self.update_figure)
        timer.start(1000)

    def compute_initial_figure(self):
        self.axes.plot([0, 1, 2, 3], [1, 2, 0, 4], 'r')

    def update_figure(self):
        # Build a list of 4 random integers between 0 and 10 (both inclusive)
        l = [random.randint(0, 10) for i in range(4)]

        self.axes.plot([0, 1, 2, 3], l, 'r')
        self.draw()


class ApplicationWindow(QtWidgets.QMainWindow):
    def __init__(self):
        QtWidgets.QMainWindow.__init__(self)
        self.setAttribute(QtCore.Qt.WA_DeleteOnClose)
        self.setWindowTitle("application main window")

        self.file_menu = QtWidgets.QMenu('&File', self)
        self.file_menu.addAction('&Quit', self.fileQuit,
                                 QtCore.Qt.CTRL + QtCore.Qt.Key_Q)
        self.menuBar().addMenu(self.file_menu)

        self.help_menu = QtWidgets.QMenu('&Help', self)
        self.menuBar().addSeparator()
        self.menuBar().addMenu(self.help_menu)

        self.help_menu.addAction('&About', self.about)

        self.main_widget = QtWidgets.QWidget(self)

        l = QtWidgets.QVBoxLayout(self.main_widget)
        sc = MyStaticMplCanvas(self.main_widget, width=5, height=4, dpi=100)
        dc = MyDynamicMplCanvas(self.main_widget, width=5, height=4, dpi=100)
        l.addWidget(sc)
        l.addWidget(dc)

        self.main_widget.setFocus()
        self.setCentralWidget(self.main_widget)

        self.statusBar().showMessage("All hail matplotlib!", 2000)

    def fileQuit(self):
        self.close()

    def closeEvent(self, ce):
        self.fileQuit()

    def about(self):
        QtWidgets.QMessageBox.about(self, "About",
                                    """embedding_in_qt5.py example
Copyright 2005 Florent Rougon, 2006 Darren Dale, 2015 Jens H Nielsen

This program is a simple example of a Qt5 application embedding matplotlib
canvases.

It may be used and modified with no restriction; raw copies as well as
modified versions may be distributed without limitation.

This is modified from the embedding in qt4 example to show the difference
between qt4 and qt5"""
                                )



# qApp = QtWidgets.QApplication
# aw = ApplicationWindow()
# aw.setWindowTitle("%s" % progname)
# aw.show()
#qApp.exec_()

# Simple window

In [2]:
#!/usr/bin/env python
"""Simple Qt4 example to manually test event loop integration.
This is meant to run tests manually in ipython as:
In [5]: %gui qt
In [6]: %run gui-qt.py
Ref: Modified from http://zetcode.com/tutorials/pyqt4/firstprograms/
"""

from PyQt5 import QtGui, QtCore, QtWidgets

class SimpleWindow(QtWidgets.QWidget):
    def __init__(self, parent=None):
        QtGui.QWidget.__init__(self, parent)

        self.setGeometry(300, 300, 200, 80)
        self.setWindowTitle('Hello World')

        quit = QtGui.QPushButton('Close', self)
        quit.setGeometry(10, 10, 60, 35)

        self.connect(quit, QtCore.SIGNAL('clicked()'),
                     self, QtCore.SLOT('close()'))

if __name__ == '__main__':
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtGui.QApplication([])

    sw = SimpleWindow()
    sw.show()

    try:
        from IPython.lib.guisupport import start_event_loop_qt4
        start_event_loop_qt4(app)
    except ImportError:
        app.exec_()

AttributeError: module 'PyQt5.QtGui' has no attribute 'QWidget'

# Old multiprocessing

In [1]:
from silq.tools.gui_tools import GUIServer
from qcodes.process.server import ServerManager

server_manager = ServerManager('gui_manager', GUIServer)

<IPython.core.display.Javascript object>

In [2]:
from qcodes.process.stream_queue import StreamQueue
stream_queue = StreamQueue.instance

In [3]:
server_manager.write('append', 'asdf')

In [4]:
stream_queue.get()

''

In [27]:
from qcodes.widgets.widgets import SubprocessWidget
SubprocessWidget.instance = None

w = SubprocessWidget.instance

In [28]:
import qcodes as qc
qc.show_subprocess_widget()

In [1]:
import silq
silq.initialize('EWJN', mode='analysis')

Initializing imports


<IPython.core.display.Javascript object>

Initializing functions
Initializing data
Initializing analysis
Initializing plotting
Initialization complete


In [1]:
from silq.tests.test_instruments import TestInstrument

test_instrument = TestInstrument('test_ins', server_name='')

<IPython.core.display.Javascript object>

In [2]:
import multiprocessing as mp
queue = mp.Queue()

p = mp.Process(target=create_gui, args=(queue,))
p.start()

In [2]:
from qcodes.process.server import ServerManager, BaseServer

In [3]:
from silq.tests.test_gui import GUIServer

server_manager = ServerManager('gui_manager', GUIServer)

In [9]:
server_manager.write('add_text', 'asdf')

In [ ]:
import sys
from PyQt4 import QtGui


class Example(QtGui.QMainWindow):
    
    def __init__(self):
        super(Example, self).__init__()
        
        self.initUI()
        
        
    def initUI(self):               
        
        textEdit = QtGui.QTextEdit()
        self.setCentralWidget(textEdit)

        exitAction = QtGui.QAction(QtGui.QIcon('exit24.png'), 'Exit', self)
        exitAction.setShortcut('Ctrl+Q')
        exitAction.setStatusTip('Exit application')
        exitAction.triggered.connect(self.close)

        self.statusBar()

        menubar = self.menuBar()
        fileMenu = menubar.addMenu('&File')
        fileMenu.addAction(exitAction)

        toolbar = self.addToolBar('Exit')
        toolbar.addAction(exitAction)
        
        self.setGeometry(300, 300, 350, 250)
        self.setWindowTitle('Main window')    
        self.show()
        
        
def main():
    
    app = QtGui.QApplication(sys.argv)
    ex = Example()
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()    